In [1]:
import requests
import pandas as pd

C:\Users\whitl\AppData\Local\Temp\ipykernel_13300\1873390105.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
mm_access_token = "da9346f92c84282b249ab83d5c2d793c"

In [3]:
# Collect the artists that performed on the main stage at Coachella
coachella_perf = pd.read_csv("data/coachella_artists.csv")
coachella_artists = coachella_perf.drop_duplicates(subset='id')
name_list = coachella_artists['name'].to_list()

main_stage = coachella_perf[coachella_perf['stage'] == "Coachella Stage"].drop_duplicates('id')['name'].to_list()
main_stage

['Beck',
 'The Chemical Brothers',
 'Morrissey',
 'Modest Mouse',
 'Tool',
 'Rage Against the Machine',
 'Pavement',
 'Thievery Corporation',
 'Mix Master Mike',
 'Paul Oakenfold',
 'Weezer',
 'The Roots',
 'MC Supernatural',
 'Iggy Pop',
 'The Dandy Warhols',
 'Souls of Mischief',
 'Medusa',
 'Ozomatli',
 'Sigur Rós',
 'Björk',
 'Siouxsie & the Banshees',
 'Jack Johnson',
 'Pete Yorn',
 'G. Love & Special Sauce',
 'KRS-One',
 'Princess Superstar',
 'Vin Rock',
 'Queens of the Stone Age',
 'Saul Williams',
 'The  Noise Conspiracy',
 'Oasis',
 'The Prodigy',
 'Foo Fighters',
 'The Strokes',
 'Saves the Day',
 'Belle & Sebastian',
 'The Mars Volta',
 'Tiësto',
 'Beastie Boys',
 'Ben Harper & the Innocent Criminals',
 'Blur',
 'The Hives',
 'The Donnas',
 'The Mooney Suzuki',
 'Kinky',
 'Red Hot Chili Peppers',
 'Iggy and the Stooges',
 'The White Stripes',
 'Sonic Youth',
 'The Polyphonic Spree',
 'The Soundtrack of Our Lives',
 'Whirlwind Heat',
 'Interpol',
 'Café Tacvba',
 'Radiohead'

In [15]:
# Search each artist from the main stage for their top 5 tracks, parse response
mm_url = f"http://api.musixmatch.com/ws/1.1/track.search"
accepted = [] # songs where the artist matches the searched artist
rejected = [] # songs where the artist does not match the searched artist
for artist in main_stage:
    search_name = artist.lower()
    # Params filters the serach to the top 5 songs by the searched artist that have lyrics data on MusixMatch (top 5 by MusixMatch's own ranking)
    params = {"apikey": mm_access_token,
              "q_artist": search_name,
              "page_size": 5,
              "page": 1,
              "f_has_lyrics": True}
    response = requests.get(mm_url, params=params)
    for track in response.json()['message']['body']['track_list']:
        # parse the response
        tdata = track['track']
        row = {"name": artist,
        "artist_name": tdata['artist_name'],
        "track_name": tdata['track_name'],
        "track_id": tdata['track_id'],
        "source": tdata}
        if track['track']['artist_name'] != artist:
            rejected.append(row)
        else:
            accepted.append(row)

In [20]:
# Writing out the searched name and the name found for tracks where they do not match
# In the discrepancies.txt file, the rows where removed manually where the searched artist was not actually a part of the track
#   EX: Paul Oakenfold vs. Paul Oakenfold feat. Andy Gray
# This was done manually because of the amount of variation in artist names and formats for artist names from Spotify vs. MusixMatch
txt = ''
for t in rejected:
    txt += f"{t['name']}, {t['artist_name']}\n"
    
with open("data/discrepancies.txt", 'w') as f:
    f.write(txt)

In [28]:
# Read the final version of the discrepancies.txt to move the tracks that are correct into the accepted list
lines = []
with open("data/discrepancies.txt") as f:
    lines = f.readlines()

pairs = []
for l in lines:
    x = l.split(', ')
    x[1] = x[1].replace('\n', '') 
    pairs.append((x[0], x[1]))

In [30]:
# Move songs from the rejected list to the accepted list if they are in the discrepancies file.
count = 0
for entry in rejected:
    if (entry['name'], entry['artist_name']) in pairs:
        accepted.append(entry)
        print(entry)
        count += 1
count

{'name': 'The Chemical Brothers', 'artist_name': 'The Chemical Brothers feat. Miguel', 'track_name': 'This Is Not A Game - From "The Hunger Games: Mockingjay Part 1" Soundtrack', 'track_id': 84369062, 'source': {'track_id': 84369062, 'track_name': 'This Is Not A Game - From "The Hunger Games: Mockingjay Part 1" Soundtrack', 'track_name_translation_list': [], 'track_rating': 39, 'commontrack_id': 47417980, 'instrumental': 0, 'explicit': 0, 'has_lyrics': 1, 'has_subtitles': 1, 'has_richsync': 0, 'num_favourite': 3, 'album_id': 20900916, 'album_name': 'The Hunger Games: Mockingjay Pt. 1 (Original Motion Picture Soundtrack)', 'artist_id': 27955516, 'artist_name': 'The Chemical Brothers feat. Miguel', 'track_share_url': 'https://www.musixmatch.com/lyrics/The-Chemical-Brothers-feat-Miguel/This-Is-Not-A-Game-From-The-Hunger-Games-Mockingjay-Part-1-Soundtrack?utm_source=application&utm_campaign=api&utm_medium=Technical+University+of+Denmark%3A1409624530551', 'track_edit_url': 'https://www.musi

429

In [31]:
mm_track_data = pd.DataFrame(accepted)
mm_track_data.to_csv('data/mm_track_metadata.csv') # save as a checkpoint

In [44]:
lyrics = []

In [50]:
# Search each artist from the main stage for their top 5 tracks, parse response
mm_url = f"http://api.musixmatch.com/ws/1.1/track.lyrics.get"
mm_track_data = pd.read_csv("data/mm_track_metadata.csv")
 # songs where the artist does not match the searched artist
track_list = mm_track_data['track_id'].to_list()
for track_id in track_list:
    print(f"Searching: {track_id}")
    params = {"apikey": mm_access_token,
              "track_id": track_id}
    response = requests.get(mm_url, params=params)
    if not response.json()['message']['body']: # check for data
       print("Skipping")
       continue

    l = response.json()['message']['body']['lyrics']
    row = {"track_id": track_id,
           "text": l['lyrics_body'],
           "explicit": l['explicit'],
           "script_tracking_url": l["script_tracking_url"]}
    lyrics.append(row)

Searching: 73423430
Searching: 33525765


Searching: 171156404
Searching: 181513284
Searching: 37148955
Searching: 37149056
Searching: 35436768
Searching: 41034257
Searching: 38273665
Searching: 231465195
Searching: 112519443
Searching: 228999081
Searching: 159291580
Searching: 186859857
Searching: 161641028
Searching: 32096269
Searching: 133575588
Searching: 254445656
Searching: 2300903
Searching: 254501711
Searching: 95091085
Searching: 204380960
Searching: 141598025
Searching: 261960614
Searching: 31613046
Searching: 84854728
Searching: 84585926
Searching: 231758842
Searching: 84461304
Searching: 86097597
Searching: 15977915
Searching: 33861746
Searching: 109898369
Searching: 16542033
Searching: 34055878
Searching: 86094684
Searching: 114134508
Searching: 84329146
Searching: 41121792
Searching: 89974441
Searching: 171682051
Searching: 44989178
Searching: 37769676
Searching: 62603005
Searching: 84372474
Searching: 95745154
Searching: 16846829
Searching: 31106599
Searching: 19446636
Searching: 17955237
Searching: 114520142
Se

TypeError: list indices must be integers or slices, not str

In [56]:
lyrics_df = pd.DataFrame(lyrics)
lyrics_df = lyrics_df.drop_duplicates(subset='track_id')

In [80]:
lyrics_df

,track_id,text,explicit,script_tracking_url
0,35147897,Your name in lights\nVibrating to your ring to...,0,https://tracking.musixmatch.com/t1.0/m_js/e_1/...
1,178403480,Your name in lights\nVibrating to your ring to...,0,https://tracking.musixmatch.com/t1.0/m_js/e_1/...
2,43666506,Do it again\nDo it again\nDo it again\nDo it a...,0,https://tracking.musixmatch.com/t1.0/m_js/e_1/...
3,179156467,Listen to your heart\nDon't run away\nListen t...,0,https://tracking.musixmatch.com/t1.0/m_js/e_1/...
4,976849,"Head in the clouds, and a mouthful of pie\nhea...",0,https://tracking.musixmatch.com/t1.0/m_js/e_1/...
...,...,...,...,...
1290,84231905,Reminicin' on our time of innocence\nWhen we d...,1,https://tracking.musixmatch.com/t1.0/m_js/e_1/...
1291,74478946,Smoke from the loosie drift\nHold it like a cr...,1,https://tracking.musixmatch.com/t1.0/m_js/e_1/...
1292,198336827,I woke up early once again that's four days st...,1,https://tracking.musixmatch.com/t1.0/m_js/e_1/...
1293,198336822,"Get a dose of dirty code to go, been cold sinc...",1,https://tracking.musixmatch.com/t1.0/m_js/e_1/...


In [66]:
artist_data = pd.read_csv("data/coachella_artists_full.csv")

In [84]:
# Adding some of the artist data (artist_name, artist_genres, artist_id) to the track dataset, formatting the table
full_track_data = lyrics_df.merge(mm_track_data, on='track_id', how='left')
full_track_data = full_track_data.drop('Unnamed: 0', axis=1)
full_track_data = full_track_data.drop('artist_name', axis=1)
full_track_data = full_track_data.merge(artist_data[['name','id' , 'genres']], on='name', how='left')
full_track_data = full_track_data.rename(columns={'name': 'artist_name',
                                            'id': 'artist_id',
                                            'genres': 'artist_genres'})
full_track_data = full_track_data.drop_duplicates(subset='track_id').reset_index()
full_track_data = full_track_data.drop('index', axis=1)

In [86]:
# Save results
full_track_data.to_csv('data/song_lyrics_dataset.csv')